In [ ]:
import os
import sys
import fnmatch
import zipfile
import xmltodict
import numpy as np
import pandas as pd
import json
import gzip
import pickle
import csv
import scipy.sparse

In [ ]:
faculty_affil = pd.read_csv('faculty-affiliations.csv')
def csv2dict_str_str(fname):
    with open(fname, mode='r') as infile:
        rdr = csv.reader(infile)
        d = {rows[0].strip(): rows[1].strip() for rows in rdr}
    return d
aliasdict = csv2dict_str_str('dblp-aliases.csv')

In [ ]:
csr_unis = faculty_affil.affiliation.unique()

In [ ]:
r1 = pd.read_csv('other_ranks/r1.csv',header=None)

In [ ]:
i = 0
for row in r1.itertuples():
    if not row[1] in csr_unis:
        print(row[1])
        i +=1
print(i)
r1 = list(r1[0])

In [ ]:
r1_faculty_affil = {k:v for k,v in faculty_affil.iteritems()}

In [ ]:
# get r1
filter_set = [1 if row[2] in r1 else 0 for row in faculty_affil.itertuples()]
fss = np.array(filter_set).astype(np.bool)
r1_faculty_affil = faculty_affil[fss]

# get alias
r1_unique = sorted(set([aliasdict.get(row[1],row[1]) for row in r1_faculty_affil.itertuples()]))
len(r1_unique),r1_faculty_affil.shape
r1_unique_dict = {k:1 for k in r1_unique}
affils_dict = {row[1]:row[2] for row in r1_faculty_affil.itertuples()} 

In [ ]:
with gzip.open('useful_venue_list.pkl.gz','rb') as fp:
    all_venues = pickle.load(fp)
with gzip.open('useful_authors_list.pkl.gz','rb') as fp:
    all_authors = pickle.load(fp)
with gzip.open('useful_papers.pkl.gz','rb') as fp:
    all_papers = pickle.load(fp)
conf_idx = {v:i for i,v in enumerate(all_venues)}
name_idx = {v:i for i,v in enumerate(all_authors)}
n_confs = len(all_venues)
n_auths = len(all_authors)
#r1_confs = pickle.load(open('old_version/r1_confs.pkl','rb'))
#r1_confs_dict = {_:1 for _ in r1_confs}

In [ ]:
Xr1 = scipy.sparse.dok_matrix((len(r1),n_confs))
xdict = {}
for paper in all_papers:
    tag,title, authors, conf, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    j = conf_idx[conf]
    if year < 2000:
        continue
    for a in authors:
        name = aliasdict.get(a,a)
        if name in affils_dict:
            uni = affils_dict[name]
            i = r1.index(uni)
            xdict[(i,j)] = 1
Xr1._update(xdict)


In [ ]:
filtered_confs = (Xr1.sum(0) > 5)



r1_confs = [conf for conf in all_venues if filtered_confs[0,conf_idx[conf]]]
print(len(r1_confs))
#print(r1_confs)
r1_conf_idx = {v:i for i,v in enumerate(r1_confs)}
r1_conf_idx_orig = [conf_idx[conf] for conf in all_venues if filtered_confs[0,conf_idx[conf]]]

r1_scores = Xr1[:,np.array(filtered_confs).reshape((-1))]
r1_scores = np.array(r1_scores.sum(0)).reshape((-1))


In [ ]:
Xpaper = scipy.sparse.dok_matrix((len(r1_unique),len(r1_confs)))
xdict = {}
for paper in all_papers:
    tag,title, authors, conf, pages, startPage,year,volume,number,url,publtype,eb_toofew,eb_skip = paper
    if conf not in r1_conf_idx:
        continue
    if year < 2000:
        continue
    j = r1_conf_idx[conf]
    for a in authors:
        name = aliasdict.get(a,a)
        if name in affils_dict:
            i = r1_unique.index(name)
            xdict[(i,j)] = 1 + xdict.get((i,j),0)
Xpaper._update(xdict)

In [ ]:
r1counts = np.squeeze(np.array(Xpaper.sum(0)))

In [ ]:
#filtered_authors = np.squeeze(np.array(Xpaper.sum(1) > 15) & np.array(Xpaper.sum(1) < 500))
#Xpaper_da = Xpaper[filtered_authors,:]
Xpaper_da = Xpaper

In [ ]:
Xpaper_da

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
clf = LatentDirichletAllocation(50)

In [ ]:
what = clf.fit_transform(Xpaper_da.T)
what.shape

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
embed = TSNE(metric='cosine')
ys = embed.fit_transform(what)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
km = MiniBatchKMeans(25,n_init=20)
km.fit(what)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.scatter(ys[:,0],ys[:,1],c=km.labels_/25,cmap='tab20c')
plt.grid(True)

In [ ]:
for i in range(25):
    tmp = [(r1_scores[i],i) for i,v in enumerate(km.labels_ == i) if v ] 
    score_idx = sorted(tmp,reverse=True)
    print(i)
    for s,idx in score_idx[:10]:
        print('\t',r1_confs[idx])
    print()

In [ ]:
pickle.dump(what,open('top_conf_embed.pkl','wb'))
pickle.dump(filtered_confs,open('top_conf_ind.pkl','wb'))

In [ ]:
pickle.dump(r1_scores,open('r1_scores.pkl','wb'))
pickle.dump(r1_confs,open('r1_confs.pkl','wb'))